# Motor Control

The telescope is remotely controlled using 2 stepper-motors. Each stepper-motor controls one degree of freedom. One motor spins the telescope controlling the Optical Tube Assembly's (OTA) compass heading, azimuth (AZ). The other controls the angle the OTA makes to the ground, altitude (ALT)

![ALT-AZ Coordinates](Altaz-coordinates-64fd7c4.webp "ALT-AZ Coordinates")


## Motor Physical Limits

Due to the physical attachment to the telescope the motors have limited movement. Understanding the use of the telescope and these physical limits is important in understanding the implementation.



### ALT

The ALT motor moves the OTA from 0 to 90 degrees in relationship to the ground. 

Because the most sites have physical limits to the horizon due to things like trees and houses. So its more like 10 to 90 degrees due to the environment. Of course that may be very direction based. 

Understanding the physics is all about the moment arm and the degree to which the scope is balanced. We can get away with a less powerful stepper-motor (Nm) the better balanced the scope is. In a Dobsonian scope you typically have the optics balanced against the mirror. Since the pivot point may not be dead center, the eyepiece optics/camera(s) have a moment arm multiplier. The largest downward force is when the scope is at 0 degrees to the horizon, when the OTA tube itself doesn't carry some of the weight of the eyepiece optics.

Based on the current configuration of the eyepiece optics, we will experimentally identify the counterweight needed to achieve 0 degrees. This should allow us to easily work what ever the horizon presents.

It is also important to note that when storing the scope it is stored ant 90 degrees to the horizon. We use the ALT = 90 degrees as the home location of the motor. The motor will allow us to move a little past 90 degrees but are physically limited by the Dobsonian mount.

### AZ

The AZ motor moves the OTA through some degrees of the compass heading. When tracking a star from horizon to horizon it can be at most 180 degrees.  

Many sites may be limited by things like a building. So you may also be limited by that.

We use a fixed belt system. This limits our motion to around 270 degrees. We will determine the maximum degree of motion experimentally. 

Also one of the questions is: What is the home position? You will always track east to west. But depending on the direction you are facing, north or south; that means moving the belt in a different direction. Facing direction is part of the equation to determine direction of tracking. So by convention, I will pick the center of motion as home. The idea is to point that scope basically north or south when at its storage location. 

Initial alignment will be by picking a well known star. Based on the time and location, the azimuth can be determined. That will be the session zero. Home location 0, will take you back to the center of motion. 

### Motion Control

Both stepper-motors are attached to a stepper-motor controller that is used to move the motor in fractions of steps. The motor controller I will use is a custom controller I have access to. The code and notebooks will expose the interface to the custom controller board. The board uses a serial connection and commands exposed to through the interface.  That would mean anyone else looking to use this code will have to substitute their controller. 

There are a lot of custom things here. Everything from the weight of the scope to the motors I use. The motors used are based on physics calculations. This notebook is about testing those equations and achieving experimental results.




## Setup

In [28]:
!python3 -m pip install pyserial

Defaulting to user installation because normal site-packages is not writeable


In [43]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

print([port.name for port in ports])

['ttyACM4', 'ttyACM0']


In [44]:
motors = serial.Serial('/dev/ttyACM4')
motors

Serial<id=0x7f47643705b0, open=True>(port='/dev/ttyACM4', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [45]:
motors.write(b'?')
return_string = str(motors.read_until(expected= b'?\tdisplay this help screen\r\n'),'utf-8')
return_string.splitlines()

['stepper_amis command set',
 'Ground Teensy pin 15 to disable joysticks for system setup',
 'f = float, n = integer, both end with space',
 'Ad\tSelect current axis to d (0 to 2)',
 'B\tRead joyMax for selected axis',
 'C\tGet motion status',
 'Dd\tSet number of microsteps per step for selected axis',
 'Ed\tEEPROM (0 = load, 1 = save, 2 = scramble',
 'e\tRun self-tests for selected axis',
 'Fd\tEnable/Disable Joystick (0 = disable, 1 = enable)',
 'f\tUpload chord',
 'G\tGet status dictionary 1 (major stuff)',
 'g\tGet status dictionary 2 (minor stuff)',
 'Hd\tLock/Unlock Joystick (0 = unlocked, 1 = locked)',
 'hd\tPrint chord d',
 'If\tSet run current of motor to f (mA)',
 'i\tRun chords on selected axis',
 'J\tRead joystick zero',
 'K\tKill motor motion always, and disable joysticks',
 'k\tKill motor motion (verbose mode for debugging)',
 'Lf\tSet hold current of motor (mA)',
 'Nd\tEnable limit switch for selected axis (0 = disable, 1 = enable)',
 'Od\tMove d relative microsteps, met

Axis 0 is the altitude. It ranges from 0 to approx 140000.

In [50]:
motors.write(b'A0')
motors.read_until(expected= b'}')

b"{'ax': 0}"

Go to Horizontal Alt

In [ ]:
motors.write(b'o140000')
motors.read_until(expected=b"{'alldone': 1}\r\n")

Home to zenith

In [16]:
motors.write(b'o0')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"{'t': 35.25}\r\n{'alldone': 1}\r\n"

In [51]:
motors.write(b'o60000')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"\r\n{'t': 30.25}\r\n{'alldone': 1}\r\n"

Axis 1 is the Azmuth. Absolute position > 0 are counter clockwise from 0. Absolute position < 0 are clockwise from 0. Safe -110000 < azmuth < 110000

In [46]:
motors.write(b'A1')
motors.read_until(expected= b'}')

b"{'ax': 1}"

In [40]:
motors.write(b'o0')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"{'t': 45.25}\r\n{'alldone': 1}\r\n"

In [47]:
motors.write(b'o-110000')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"\r\n{'t': 27.75}\r\n{'alldone': 1}\r\n"

In [48]:
motors.write(b'o110000')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"{'t': 55.25}\r\n{'alldone': 1}\r\n"

Home

In [52]:
motors.write(b'A0')
motors.read_until(expected= b'}')
motors.write(b'o0')
motors.read_until(expected=b"{'alldone': 1}\r\n")
motors.write(b'A1')
motors.read_until(expected= b'}')
motors.write(b'o0')
motors.read_until(expected=b"{'alldone': 1}\r\n")

b"\r\n{'t': 15.25}\r\n{'alldone': 1}\r\n"